In [19]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import sys, re, os
import numpy as np
import pandas as pd
import glob,imp
from pandas import json_normalize
import pyslha
sys.path.append('../../smodels')
from smodels.particlesLoader import getParticlesFromSLHA
pd.options.mode.chained_assignment = None #Disable copy warnings
from smodels.experiment.databaseObj import Database
from smodels.base.model import Model
from smodels.decomposition import decomposer
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.base.physicsUnits import fb, GeV
#from smodels.share.models.ppZpjj import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.smodelsLogging import setLogLevel
from smodels.base import runtime
from smodels.experiment.expSMS import ExpSMS
from smodels.experiment.defaultFinalStates import finalStates
setLogLevel("info")
#runtime.modelFile = 'smodels.share.models.ppZpjj'
!runSModelS.py -V

3.0.0-beta


In [20]:
BSMlist = getParticlesFromSLHA('TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha')
model = Model(BSMparticles=BSMlist, SMparticles=SMList)
#model.updateParticles(inputFile=slhaFile)
print(BSMlist)

[xd, xd~, y1]


In [21]:
slhaFolder = 'TRV1tt_slha'
resultsFolder = 'TRV1tt_results'
File = 'parameters.ini'
!chmod -R +rwx {slhaFolder}

In [22]:
!runSModelS.py -p {File} -f {slhaFolder} -o {resultsFolder}

INFO in databaseObj.loadBinaryFile() in 542: loading binary db file /home/yoxara/smodels-database/db3.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 549: Loaded database from /home/yoxara/smodels-database/db3.pcl in 2.3 secs.
INFO in modelTester.loadDatabaseResults() in 497: Including non-validated results
INFO in modelTester.testPoints() in 357: Running SModelS for 423 files with a single process. Messages will be redirected to smodels.log


In [23]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(resultsFolder+'/*.py'):
    with open(f,'r') as ff:
        dataF = ff.read()
    dataF = dataF.replace('inf','-1')
    with open(f.replace('.py','_fix.py'),'w') as ff:
        ff.write(dataF)
    f = f.replace('.py','_fix.py')
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)

    data.append(dataDict)
    

In [24]:
print(len(data))

423


In [25]:
#Convert data to flat DataFrame:
pd.set_option('display.float_format', '{:e}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 7)
smodelsDF = json_normalize(data)
smodelsDF

filename  \
0    run_423_MZp_3000_gAq_3.000000E-01.slha   
1    run_365_MZp_2800_gAq_1.297300E-01.slha   
2    run_341_MZp_2700_gAq_2.000000E-01.slha   
..                                      ...   
420  run_667_MZp_3900_gAq_5.000000E-01.slha   
421  run_855_MZp_4600_gAq_3.000000E-01.slha   
422  run_552_MZp_3500_gAq_1.100000E-01.slha   

                                     SMS Decomposition  \
0    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
1    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
2    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
..                                                 ...   
420  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
421  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
422  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   

     Total xsec for missing topologies (fb)  \
0                              7.152890e+01   
1                              2.100330e+01   
2                              6.295194e+01   
..                                      ...   
420                            2.877249e+01   
421                            2.414649e+00   
422                            3.241948e+00   

     Total xsec for missing topologies with displaced decays (fb)  \
0                                         0.000000e+00              
1                                         0.000000e+00              
2                                         0.000000e+00              
..                                                 ...              
420                                       0.000000e+00              
421                                       0.000000e+00              
422                                       0.000000e+00              

     Total xsec for missing topologies with prompt decays (fb)  \
0                                         8.555351e+01           
1                                         2.510923e+01           
2                                         7.523706e+01           
..                                                 ...           
420                                       3.445997e+01           
421                                       2.893534e+00           
422                                       3.880966e+00           

     Total xsec for topologies outside the grid (fb)  OutputStatus.sigmacut  \
0                                       0.000000e+00           0.000000e+00   
1                                       0.000000e+00           0.000000e+00   
2                                       0.000000e+00           0.000000e+00   
..                                               ...                    ...   
420                                     0.000000e+00           0.000000e+00   
421                                     0.000000e+00           0.000000e+00   
422                                     0.000000e+00           0.000000e+00   

     OutputStatus.minmassgap  OutputStatus.maxcond  OutputStatus.ncpus  \
0                         50          2.000000e-01                   1   
1                         50          2.000000e-01                   1   
2                         50          2.000000e-01                   1   
..                       ...                   ...                 ...   
420                       50          2.000000e-01                   1   
421                       50          2.000000e-01                   1   
422                       50          2.000000e-01                   1   

                                    OutputStatus.model  \
0    TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha   
1    TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha   
2    TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha   
..                                                 ...   
420  TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha   
421  TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha   
422  TRV1tt_slha/run_91_MZp_1800_gAq_8.200000E-02.slha   

     OutputStatus.promptwidth  OutputStatus

In [26]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key >= 5000001])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['DMINPUTS'].items()])
    #extparDict.update(dict([[str(key),val] for key,val in slha.blocks['BLINPUTS'].items()]))
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key >= 5000001])
    BRsDict = {}
    for pdg,val in slha.decays.items():
#         if not abs(pdg) in [1000024,1000023]:
#             continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)


In [27]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [28]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')

In [29]:
print('Final number of data points:',dataDF.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

Final number of data points: 423


In [30]:
#Save DataFrame to pickle file:
dataDF.to_pickle('TRV1tt_data.pcl')
print(len(dataDF))
dataDF

423


filename  mass.5000001  mass.5000521  \
0    run_423_MZp_3000_gAq_3.000000E-01.slha  3.000000e+03  1.000000e+04   
1    run_365_MZp_2800_gAq_1.297300E-01.slha  2.800000e+03  1.000000e+04   
2    run_341_MZp_2700_gAq_2.000000E-01.slha  2.700000e+03  1.000000e+04   
..                                      ...           ...           ...   
420  run_667_MZp_3900_gAq_5.000000E-01.slha  3.900000e+03  1.000000e+04   
421  run_855_MZp_4600_gAq_3.000000E-01.slha  4.600000e+03  1.000000e+04   
422  run_552_MZp_3500_gAq_1.100000E-01.slha  3.500000e+03  1.000000e+04   

     width.5000001  width.5000521     extpar.1     extpar.2     extpar.3  \
0     2.569856e-02   0.000000e+00 0.000000e+00 1.500000e+00 3.000000e-01   
1     4.483046e-03   0.000000e+00 0.000000e+00 1.500000e+00 1.297300e-01   
2     1.027152e-02   0.000000e+00 0.000000e+00 1.500000e+00 2.000000e-01   
..             ...            ...          ...          ...          ...   
420   9.292476e-02   0.000000e+00 0.000000e+00 1.500000e+00 5.000000e-01   
421   3.947884e-02   0.000000e+00 0.000000e+00 1.500000e+00 3.000000e-01   
422   4.034376e-03   0.000000e+00 0.000000e+00 1.500000e+00 1.100000e-01   

        extpar.4  xsec13TeV(fb).5000001   BRs.y1.q,q   BRs.y1.c,c  \
0   0.000000e+00           8.555350e+01 1.672149e-01 1.672148e-01   
1   0.000000e+00           2.510923e+01 1.672961e-01 1.672959e-01   
2   0.000000e+00           7.523705e+01 1.673436e-01 1.673433e-01   
..           ...                    ...          ...          ...   
420 0.000000e+00           3.445997e+01 1.669911e-01 1.669910e-01   
421 0.000000e+00           2.893533e+00 1.668999e-01 1.668998e-01   
422 0.000000e+00           3.880966e+00 1.670695e-01 1.670694e-01   

      BRs.y1.b,b  BRs.y1.t-,t+  totalxsec13TeV(fb)  totalxsec8TeV(fb)  \
0   1.672125e-01  1.639279e-01        8.555350e+01       0.000000e+00   
1   1.672932e-01  1.635227e-01        2.510923e+01       0.000000e+00   
2   1.673405e-01  1.632855e-01        7.523705e+01       0.000000e+00   
..           ...           ...                 ...                ...   
420 1.669896e-01  1.650461e-01        3.445997e+01       0.000000e+00   
421 1.668988e-01  1.655018e-01        2.893533e+00       0.000000e+00   
422 1.670677e-01  1.646545e-01        3.880966e+00       0.000000e+00   

                                     SMS Decomposition  \
0    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
1    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
2    [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
..                                                 ...   
420  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
421  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   
422  [{'ID': 1, 'SMS': '(PV > y1)', 'Masses (GeV)':...   

     Total xsec for missing topologies (fb)  \
0                              7.152890e+01   
1                              2.100330e+01   
2                              6.295194e+01   
..                                      ...   
420                            2.877249e+01   
421                            2.414649e+00   
422                            3.241948e+00   

     Total xsec for missing topologies with displaced decays (fb)  \
0                                         0.000000e+00              
1                                         0.000000e+00              
2                                         0.000000e+00              
..                                                 ...              
420                                       0.000000e+00              
421                                       0.000000e+00              
422                                       0.000000e+00              

     Total xsec for missing topologies with prompt decays (fb)  \
0                                         8.555351e+01           
1                                         2.510923e+01           
2                                         7.523706e+01           
..                          

In [31]:
dataDF['xsec13TeV(fb).5000001'] * (dataDF['BRs.y1.t-,t+'])




0     1.402461e+01
1     4.105929e+00
2     1.228512e+01
          ...     
420   5.687484e+00
421   4.788850e-01
422   6.390185e-01
Length: 423, dtype: float64

In [32]:
dataDF['ExptRes.result0.theory prediction (fb)']

0     1.402461e+01
1     4.105929e+00
2     1.228512e+01
          ...     
420   5.687484e+00
421   4.788850e-01
422   6.390185e-01
Name: ExptRes.result0.theory prediction (fb), Length: 423, dtype: float64

In [33]:
dataDF['ExptRes.result0.r']

0     1.865952e+00
1     5.884899e-01
2     1.730415e+00
          ...     
420   2.252736e+00
421   2.197395e-01
422   1.327337e-01
Name: ExptRes.result0.r, Length: 423, dtype: float64

In [34]:
dataDF['ExptRes.result0.theory prediction (fb)']/dataDF['ExptRes.result0.upper limit (fb)']

0     1.865952e+00
1     5.884899e-01
2     1.730416e+00
          ...     
420   2.252737e+00
421   2.197396e-01
422   1.327337e-01
Length: 423, dtype: float64